In [ ]:
import json
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import string

# process the tweets
open the tweets, process them, then save them back into the json format

In [ ]:
# open files to read and write from
in_filename = 'SampleData.json'
out_filename = 'prepared_json.json'

in_handle = open(in_filename, encoding='utf8')
out_handle = open(out_filename, 'w', encoding='utf8')

In [54]:
# process the tweet and adds in negation terms
# adds a not to every word after a token of logical negation
# until the next puncutation mark

negation_tokens = ['not', "n't", 'no', 'never'] # note: order is important not should be first
puncuation = {',', '.', '?', '!'}

# adapted from https://stackoverflow.com/questions/23384351/how-to-add-tags-to-negated-words-in-strings-that-follow-not-no-and-never
def check_and_negate(tweet: str):
    tweet = re.sub('n\'t', ' not', tweet) # replace n't with not
    # add NOT_ to the beginning of each word until a end of sentence mark (.,!?) occurs
    tweet = re.sub(r'\b(?:not|never|no)\b[\'\w\s]+[.,?!]', 
                   lambda match: re.sub(r'(\s+)(\w+)', r'\1NOT_\2', match.group(0)), 
                   tweet,
                   flags=re.IGNORECASE)
    return tweet

In [ ]:
# processes the tweet
# https://towardsdatascience.com/the-real-world-as-seen-on-twitter-sentiment-analysis-part-one-5ac2d06b63fb
def process_tweet(tweet: str):
    tweet = tweet.lower() # convert to lowercase
    tweet = re.sub(r'#\w*', '', tweet) # remove hashtags
    tweet = re.sub('@\S+', '', tweet) # remove @username
    tweet = re.sub('\s+', ' ', tweet) # remove multiple whitespace
    tweet.strip() # remove excess leading and trailing whitespace
    tweet = check_and_negate(tweet)# add negations
    tweet.strip() # remove excess leading and trailing whitespace
    return tweet

In [ ]:
j_items = []

for line in in_handle:
    # convert json object into python dict, append to list
    item = json.loads(line)
    j_items.append(item)
    # perform cleaning and preparation
    if 'text' in item: # check that the dictionary contains the key
        item['text'] = process_tweet(item['text']) # convert to lowercase
    # write the json object into an output file
    out_handle.write(json.dumps(item))
    out_handle.write('\n')

In [ ]:
in_handle.close()
print('input file closed')
out_handle.close()
print('output file closed')

In [ ]:
# Print out first tweet with formatting
print(json.dumps(j_items[0], indent=4))

In [ ]:
print(json.dumps(j_items[2], indent=4))

In [ ]:
print(j_items[2]['text'])
print(process_tweet(j_items[2]['text']))

In [56]:
# test the negation
test = "i don't like onions he's body. They taste bad"
print(check_and_negate(test))

i do not NOT_like NOT_onions NOT_he's NOT_body. They taste bad
